In [1]:

import tensorflow_datasets as tfds


data = tfds.load("movielens/1m-ratings")

/com.docker.devenvironments.code/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-03 03:40:57.659102: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-03 03:40:57.660410: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 03:40:57.686725: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-03 03:40:57.688700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in perform

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...: 100%|██████████| 1/1 [00:11<00:00, 11.03s/ url]

Dl Completed...: 100%|██████████| 1/1 [00:11<00:00, 11.18s/ url]


Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-ratings/0.1.1. Subsequent calls will reuse this data.


In [3]:
df = tfds.as_dataframe(data["train"])
print(df.head(5))

   bucketized_user_age movie_genres movie_id                      movie_title  \
0                 35.0       [0, 7]  b'3107'              b'Backdraft (1991)'   
1                 25.0          [7]  b'2114'         b'Outsiders, The (1983)'   
2                 18.0      [4, 15]   b'256'                 b'Junior (1994)'   
3                 18.0      [0, 10]  b'1389'               b'Jaws 3-D (1983)'   
4                 18.0          [0]  b'3635'  b'Spy Who Loved Me, The (1977)'   

    timestamp  user_gender  user_id  user_occupation_label  \
0   977432193         True   b'130'                     18   
1   965932967        False  b'3829'                      0   
2  1012103552        False  b'1265'                     21   
3   972004605         True  b'2896'                     14   
4   961180111         True  b'5264'                     17   

      user_occupation_text  user_rating user_zip_code  
0   b'technician/engineer'          5.0      b'50021'  
1     b'academic/educator'  

In [4]:
# In this tutorial, let us only use the bare minimum: user_id, movie_id,
# and user_rating since very often this is the only data we have
filtered_data = (
    df
    .filter(["timestamp", "user_id", "movie_id", "user_rating"])
    .sort_values("timestamp")
    .astype({"user_id": int, "movie_id": int, "user_rating": int}) # nicer types
    .drop(columns=["timestamp"]) # don't need the timestamp anymore
)

# We will also keep the timestamp to conduct a temporal train-test split since 
#  this resembles how we train in real life: we train now, but we want the model
# to work well tomorrow. So we should evaluate the model quality like this as well.
train = filtered_data.iloc[:900000] # chronologically first 90% of the dataset
test = filtered_data.iloc[900000:]  # chronologically last 10% of the dataset

In [5]:
# cold start problem exists, meaning that some users or movies are only 
# present in the test set, but not in the training set.
print(train.query("user_id == 1").shape[0])
print(test.query("user_id == 1").shape[0])

0
53


In [8]:
# save for later in the train.py script

import os
if os.path.isdir("../local") == False:
    os.mkdir("../local")

train.to_csv("../local/train.csv", index=False)
test.to_csv("../local/test.csv", index=False)